### make_intro_figure

adapted from the pygmtexamplewitheverything/make_intro_figure.py file sent by Magali in email 04/2024

note on the files:

    the .txt files need to be under the local directory

    the .grd files from slab2.0 is stored some elsewhere

In [ ]:
#!/opt/miniconda3/bin/python
import pygmt
import json	# for reading bird plate-boundary data
import numpy as np
import os
import sys
from os.path import exists as file_exists
#import slab2bird as sb   # loads slab2bird dictionary

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

In [ ]:
# For plot read in topography for basemap and choose fonts
pygmt.config(FONT='10p,Times-Roman,black')
pygmt.config(FONT_LABEL='10p,Times-Roman,black')
pygmt.config(FONT_TITLE='10p,Times-Roman,black')
pygmt.config(MAP_TITLE_OFFSET='6.0p')
pygmt.config(FORMAT_GEO_MAP="ddd")

In [ ]:
# Read in Bird's plate boundary data from a JSON format file
# format of json file:
# It contains a "feature" field on the top level
#   "features": []
# then every "feature" contains
#   "properties": {}
# There are subfields like "Name", "Source", "Type" in the "properties" field
# the other component of the "feature" is
#   "geometry": { "type": "LineString", "coordinates": [ [ -0.437900, -54.851800 ], ... ]
# this contains the specific coorinate pairs for the geometry
dir = os.path.join(ASPECT_LAB_DIR, 'tests/integration/big_fixtures/GMT')
boundaries_json_file = os.path.join(dir, 'PB2002_boundaries.json')
file_object = open(boundaries_json_file,'r')
birddata = json.load(file_object)

In [ ]:
# Get GRD file for Slab2.0 depth data 
loc1 = 'cas'  # Cascadia
slab2dir = os.path.join(ASPECT_LAB_DIR, "Subduction-Dynamics-Tools/Jupyter-Notebooks/PyGMT-Examples/Slab2GRDFiles")
assert(os.path.isdir(slab2dir))
grdfile = slab2dir + loc1 + '_slab2_dep.grd'

In [ ]:
# Set Regional Map dimensions
west = -143.0
east = -100.25
south = 32.0
north = 58.0
region_data = np.array([west, east, south, north])

dlon_edge = 0.35
dlat_edge = 1.0
region1 = str(west) + '/' + str(east) + '/' + str(south) + '/' + str(north) 
print('Region: ', region1)

In [ ]:
# Get center position and two latitude locations to define regional projection
# this step is needed that both the centre values and the lat1, lat2 values need to be integars
# also by mutliplying 0.5, we are getting only half the length in the latitude dimension
clon = np.floor(region_data[0:2].mean())
clat = np.floor(region_data[2:4].mean())
lat1 = np.floor(0.5*(north - clat) + clat )
lat2 = np.floor(clat - 0.5*(clat - south) )

mapwidth = 4.5 # inches, sets size of figure on page
proj1 = 'B' + str(clon) + '/' + str(clat) + '/' + str(lat1) + '/' + str(lat2) + '/' + str(mapwidth) + 'i'
print('proj1: ', proj1)

In [ ]:
# Start figure
# the pygmt.datasets contains data to be imported directly (e.g. earth_relief, earth_age)
# resolution: 30s (1/120 deg)
# order of the plot: first create the basemap (fig.basemap), then specific types of plots (e.g. color map - fig.grdimage)
fig = pygmt.Figure()
fig.basemap(region=region1, projection=proj1, frame=["a5f2.5", f'WSne'])
print('Plotting topo grid')
topogrid = pygmt.datasets.load_earth_relief(region=region1,resolution="30s",registration="pixel")
fig.grdimage(grid=topogrid,cmap="gray")
fig.show()

In [ ]:
# pygmt.grd2cpt: turns a cmap (example is roma) to a format gmt could use (age.cpt) and fits the data range
# layers of the plot: newer plots will be on top of older plots (e.g. the age map here is on top of the relief map in the last section)
print('Plotting age contours')
agegrid = pygmt.datasets.load_earth_age(region=region1,resolution="10m",registration="gridline")
agecptfile ='age.cpt'
pygmt.grd2cpt(grid=agegrid,cmap='roma',output='age.cpt')
fig.grdimage(grid=agegrid,nan_transparent=True,cmap=agecptfile)
fig.grdcontour(grid=agegrid,interval=3,pen='0.5p,black')
fig.colorbar(cmap=agecptfile,position="JML+o0.5c/1.5c+w5c/0.3c", box=False, frame=["a5g5","x+lAge", "y+lmyr"])
fig.show()

In [ ]:
# Cascadia Terranes
# Here, data could be directly plotted on to the map.
# For example franciscan.txt is a .txt file containing 2 columns (lon, lat)
fig.plot(data=os.path.join(dir, 'franciscan.txt'),fill='lightblue')
fig.plot(data=os.path.join(dir, 'klamath.txt'),fill='darkblue')
fig.plot(data=os.path.join(dir, 'Siletz.txt'),fill='purple')
fig.plot(data=os.path.join(dir, 'olympic_franciscan.txt'),fill='lightbrown')
fig.plot(data=os.path.join(dir, 'Wrangellia.txt'),fill='green2',transparency="80")
fig.show()

In [ ]:
# Add the slab 2.0 data
# pygmt.makecpt: the series=(-700,0,50) option works like the arange function in numpy
# contour color not showing (magali: this doesn't work)
print('Plotting slab contours')
slabcptfile ="slabdepth.cpt"
pygmt.makecpt(cmap="buda",series=(-700,0,50),output=slabcptfile)
depth_grdfile = os.path.join(slab2dir, loc1 + '_slab2_dep.grd')
#fig.grdimage(grid=depth_grdfile,nan_transparent=True,cmap=slabcptfile)
fig.grdcontour(grid=depth_grdfile,interval=slabcptfile,pen='0.5p,black')
# add a colorbar for depth    
fig.colorbar(cmap=slabcptfile, position="JMR+o-0.5c/2c+w4c/0.3c", box=False, frame=["a100g50","x+lDepth", "y+lkm"])
fig.show()

In [ ]:
# Get Pacific-North America Plate boundaries that are not subduction zones
# SAF, Queen Charlotte, Baja California.
# the "pen" entry for "plot": adjust the color, width of line plot
#	139.75/53.5/9.7501 - these are the rgb colors (qs Magali)
#   f0.2c/0.1c+r+t - this is used to make triangles 
for i in range(len(birddata["features"])):
	if 'PA' in birddata["features"][i]["properties"].values():
		if 'NA' in birddata["features"][i]["properties"].values():
			if 'subduction' not in birddata["features"][i]["properties"].values():
				print('PA-NA', birddata["features"][i]["properties"].values())
				coords = np.array(birddata["features"][i]["geometry"]["coordinates"])
				fig.plot(x=coords[:,0],y=coords[:,1], pen="1p,darkgreen")
	# Get Juan de Fuca plate Boundaries
	# Not Subduction zones
	if 'JF' in birddata["features"][i]["properties"].values():
		if 'subduction' not in birddata["features"][i]["properties"].values():
			print('Not Subduction', birddata["features"][i]["properties"].values())
			coords = np.array(birddata["features"][i]["geometry"]["coordinates"])
			fig.plot(x=coords[:,0],y=coords[:,1], pen="1p,139.75/53.5/9.7501")
	# The Subduction zone
	if 'JF' in birddata["features"][i]["properties"].values():
		if 'subduction' in birddata["features"][i]["properties"].values():
			print('subduction', birddata["features"][i]["properties"].values())
			coords = np.array(birddata["features"][i]["geometry"]["coordinates"])
			fig.plot(x=coords[:,0],y=coords[:,1], style="f0.2c/0.1c+r+t", fill="black")
fig.show()

In [ ]:
# plot the coast line: coast function
fig.coast(shorelines="1/0.25p,white",resolution="i")
fig.show()

In [ ]:
pdffile = 'cascadia_intro_fig.pdf'
fig.savefig(pdffile)